In [ ]:
### imports

# external modules
import matplotlib.pyplot as plt
import matplotlib as mpl
import importlib

# internal modules
import datagen
importlib.reload(datagen)
import count_objects
importlib.reload(count_objects)

In [ ]:
### generate an image and count the objects in it

# generate image
image_size = (256,256)
objects = {1:10, 2:5}
im = datagen.generate_image( image_size=image_size, objects=objects )

# count objects
object_coords = count_objects.count_objects_simple(im)

# compare counts to true values
ntrue = sum(objects.values())
nreco = len(object_coords)
print('True number of objects: {}'.format(ntrue))
print('Reconstructed number of objects: {}'.format(nreco))

# make a plot
fig,ax = plt.subplots( figsize=(12,12) )
ax.imshow( im, cmap='gray' )
for coord in object_coords:
    boxhalfwidth = 5
    patchxcoord = coord[1]
    patchycoord = coord[0]
    anchor = (patchxcoord-boxhalfwidth,patchycoord-boxhalfwidth)
    box = mpl.patches.Rectangle(anchor, 2*boxhalfwidth, 2*boxhalfwidth,
                               edgecolor='r', facecolor=(0,0,0,0))
    ax.add_patch(box)